In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Load dataset
file_path = '/content/Caps.csv'
data = pd.read_csv(file_path)

# Simulate user IDs (for demonstration purposes)
np.random.seed(42)
data['user_id'] = np.random.randint(0, 100, data.shape[0])

# Encoding categorical features
user_encoder = LabelEncoder()
data['user_id'] = user_encoder.fit_transform(data['user_id'])
resto_encoder = LabelEncoder()
data['resto_id'] = resto_encoder.fit_transform(data['resto_name'])

# Preparing features and labels
X = data[['user_id', 'resto_id']]
y = data['rating']

# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining the model
num_users = len(data['user_id'].unique())
num_restaurants = len(data['resto_name'].unique())
embedding_size = 50

# User embedding
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, name='user_embedding')(user_input)
user_vec = Flatten(name='flatten_user')(user_embedding)

# Restaurant embedding
resto_input = Input(shape=(1,), name='resto_input')
resto_embedding = Embedding(input_dim=num_restaurants, output_dim=embedding_size, name='resto_embedding')(resto_input)
resto_vec = Flatten(name='flatten_resto')(resto_embedding)

# Concatenate user and restaurant vectors
concat = Concatenate()([user_vec, resto_vec])
dense = Dense(128, activation='relu')(concat)
dense = Dense(64, activation='relu')(dense)
output = Dense(1)(dense)

model = Model([user_input, resto_input], output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Training the model
history = model.fit([X_train['user_id'], X_train['resto_id']], y_train,
                    epochs=100, batch_size=64,
                    validation_data=([X_val['user_id'], X_val['resto_id']], y_val))

# Evaluating the model
val_loss, val_mae = model.evaluate([X_val['user_id'], X_val['resto_id']], y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation MAE: {val_mae}")

# Saving the model
model.save('restaurant_recommendation_model.h5')

# Function to recommend restaurants for a given user
def recommend_restaurants(user_id, top_n=10):
    user_index = user_encoder.transform([user_id])[0]
    resto_indices = np.arange(num_restaurants)
    predicted_ratings = model.predict([np.full(num_restaurants, user_index), resto_indices])
    top_indices = np.argsort(predicted_ratings[:, 0])[-top_n:][::-1]
    recommended_resto_ids = resto_encoder.inverse_transform(top_indices)
    return recommended_resto_ids

# Example recommendation
user_id = data['user_id'].iloc[0]  # Use an existing user ID from the dataset
recommendations = recommend_restaurants(user_id)
print(f"Rekomendasi untuk pengguna {user_id}:")
print(recommendations)


Epoch 1/100
7/7 [==============================] - 1s 39ms/step - loss: 18.8476 - mae: 4.3323 - val_loss: 17.9567 - val_mae: 4.2286
Epoch 2/100
7/7 [==============================] - 0s 8ms/step - loss: 17.3272 - mae: 4.1532 - val_loss: 16.3319 - val_mae: 4.0320
Epoch 3/100
7/7 [==============================] - 0s 11ms/step - loss: 15.2557 - mae: 3.8939 - val_loss: 13.9747 - val_mae: 3.7279
Epoch 4/100
7/7 [==============================] - 0s 9ms/step - loss: 12.0953 - mae: 3.4618 - val_loss: 10.4690 - val_mae: 3.2211
Epoch 5/100
7/7 [==============================] - 0s 12ms/step - loss: 7.4738 - mae: 2.6977 - val_loss: 5.7550 - val_mae: 2.3643
Epoch 6/100
7/7 [==============================] - 0s 11ms/step - loss: 2.3276 - mae: 1.3909 - val_loss: 1.4960 - val_mae: 1.0909
Epoch 7/100
7/7 [==============================] - 0s 10ms/step - loss: 0.6431 - mae: 0.6327 - val_loss: 0.4867 - val_mae: 0.5123
Epoch 8/100
7/7 [==============================] - 0s 9ms/step - loss: 0.8213 - mae:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
